In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [4]:
web = requests.get('https://www.propertypro.ng/property-for-rent/flat-apartment/?search=&bedroom=&min_price=&max_price=3000000')

data = BeautifulSoup(web.content, 'html5lib')

In [6]:
data

<!DOCTYPE html>
<html lang="en"><head>
<title>Find Flat Apartment For rent  | Rent Flat Apartment  | 91171 Available | PropertyPro.ng</title>

<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>

<link href="/assets/image/42484569b600758112d45bad3d876f78-favicon.ico" rel="icon" type="image/ico"/>
<meta content="index, follow" name="robots"/>
<link href="https://www.propertypro.ng/property-for-rent/flat-apartment" rel="canonical"/>
<meta content="Find Flat Apartment For rent  with PropertyPro.ng. Browse the Nigeria's largest property database and find Flat Apartment For rent  from the leading estate agents in Nigeria" name="description"/>
<meta content="Find Flat Apartment For rent  | Rent Flat Apartment  | 91171 Available" property="og:title"/>
<meta content="Find Flat Apartment For rent  with PropertyPro.ng. Browse the Nigeria's largest property database and find Flat Apartment For rent  from the leading estate agents in Nigeria" property="og:

In [7]:
listing_divs = data.select('div[class=single-room-sale\ listings-property]')
print("Number of apartment listings on a page:", len(listing_divs))

Number of apartment listings on a page: 20


In [8]:
listing_divs[0]

<div class="single-room-sale listings-property">
<div class="single-room-img result-img">
<a class="single-room-img-a" href="/property/3-bedroom-flat-apartment-for-rent-lekki-scheme-2-ajah-lagos-2FTMH"> <img alt="3 bedroom Flat / Apartment for rent Lekki Scheme 2 Ajah Lagos" class="listings-img" src="https://images.propertypro.ng/medium/well-finished-3-bedroom-flat-7rDarvPqTPUMYMKOV7T7.jpeg" title="3 bedroom Flat / Apartment for rent Lekki Scheme 2 Ajah Lagos"/> </a>
<div class="save-heart">
<a href="/profile/add-favorite/797373"><i class="fa fa-heart"></i></a>
</div>
<div class="room-icons">
<span><span>10</span> <img alt="icon" src="/assets/assets/img/property/2f8b655773948c43f77e8c928e94bbe3-img.png"/> </span>
</div>

</div>
<div class="single-room-text">
<a href="/property/3-bedroom-flat-apartment-for-rent-lekki-scheme-2-ajah-lagos-2FTMH"> <h2 class="listings-property-title">3 BEDROOM FLAT / APARTMENT FOR RENT</h2> </a>
<h4><img alt="icon" src="/assets/assets/img/resultpage/a6b70a0

In [9]:
listing_divs[0].select('h4')[0].text

'Lekki Scheme 2 Ajah Lagos'

In [13]:
listing_divs[0].select('h3[class*=listings-price]')[0].text.strip().replace('/year', '')

'₦ 1,500,000'

In [11]:
listing_divs[0].select('div[class*=fur-areea]')[0].text.strip().split('\n')

['3 beds', '3 baths', '4 Toilets']

In [14]:
listing_divs[7].select('div[class*=furnished-btn]')[0].text.strip().replace('\n', ' ')

'Serviced'

In [15]:
listing_divs[7].select('div[class*=result-list-details]')[0].p.text.strip().replace('Read more', '').replace('FOR RENT: ', '')

'CW02186 We are Social! Check out other Amazing premium Listings on our Social.. '

In [24]:
def parse_listing_data(location, max_price, rows_of_data=2000):
    all_listings_data = []
    page_num = 0
    
    while page_num < (rows_of_data / 20):
        url = "https://www.propertypro.ng/property-for-rent/flat-apartment/?search=" + location \
              + "&bedroom=&min_price=&max_price=" + str(max_price) + "&page=" + str(page_num)
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html5lib')
        listing_divs = soup.select('div[class=single-room-sale\ listings-property]')
        if len(listing_divs) == 0:
            break 
            
        for indv_listing in listing_divs:
            address = indv_listing.select('h4')[0].text
            price_per_Year = indv_listing.select('h3[class*=listings-price]')[0].text.strip().replace('/year', '')
            listing_data = [address, price_per_Year]
            utilities = indv_listing.select('div[class*=fur-areea]')[0].text.strip().split('\n')
            listing_data.extend(utilities)
            description = indv_listing.select('div[class*=result-list-details]')[0].p.text.strip().replace('Read more', '').replace('FOR RENT: ', '')
            extra = indv_listing.select('div[class*=furnished-btn]')[0].text.strip().replace('\n', ' ')
            listing_data.append(extra + description)
            all_listings_data.append(listing_data)
        page_num += 1    
    listing_df = pd.DataFrame(all_listings_data, columns=['Area', 'price_per_Year', 'beds', 'baths', 'toilets', 'extra'])
    
    listing_df.to_csv("rent_listings_"+location+".csv", index=False)
    return listing_df

In [25]:
lagos_data = parse_listing_data('lagos', 2500000, 5000)
lagos_data

,Area,price_per_Year,beds,baths,toilets,extra
0,Lekki Scheme 2 Ajah Lagos,"₦ 1,500,000",3 beds,3 baths,4 Toilets,Well Finished 3-Bedroom Flat; Location: LekkiS...
1,LBS Ajah Ajah Lagos,"₦ 1,000,000",3 beds,3 baths,3 Toilets,Furnished Serviced Newly Built3 bedroom apartm...
2,Tope Canaan Estate Ajah Lagos,"₦ 900,000",1 beds,2 baths,2 Toilets,A very sweet mini flat for rent at Canaan esta...
3,Ajah Lagos,"₦ 600,000",2 beds,2 baths,2 Toilets,(I.T) Newly built 2 Bedroom Flat at Ajah Igond...
4,"River View Estate After OPIC Building, Isheri ...","₦ 600,000",2 beds,2 baths,3 Toilets,Newly BuiltThe property is a newly built build...
...,...,...,...,...,...,...
4995,Ikate Lekki Lagos,"₦ 2,200,000",2 beds,2 baths,3 Toilets,Lovely and standard 2 bedroom flat at ikate le...
4996,"K&S Onitire, Abaranje Road, Ikotun Lagos Alimo...","₦ 400,000",0 beds,0 baths,0 Toilets,Fantastic and tastefully built 3bedroom flat t...
4997,Ifako Ogba Ogba Lagos,"₦ 550,000",1 beds,2 baths,2 Toilets,Newly BuiltJUST OUT! Brand New Miniflat at Ogb...
4998,Akoka Yaba Lagos,"₦ 550,000",beds,1 baths,1 Toilets,Serviced Newly BuiltNewly built & Serviced Lux...


In [26]:
ibadan_data = parse_listing_data('oyo', 3000000, 2000)
ibadan_data


,Area,price_per_Year,beds,baths,toilets,extra
0,Akala Express Ibadan Oyo,"₦ 800,000",3 beds,3 baths,3 Toilets,"3 bedroom flat at Progressive Estate, off Akal..."
1,Anfani Challenge Ibadan Oyo,"₦ 620,000",2 beds,2 baths,2 Toilets,"To Let 2 Bedroom Flats at College Crescent, An..."
2,Challenge Ibadan Oyo,"₦ 1,350,000",3 beds,3 baths,3 Toilets,To Let 3 Bedroom flats at Lagelu estate (G.R.A...
3,Oluyole Ibadan Oyo,"₦ 1,500,000",3 beds,3 baths,3 Toilets,Newly built 3 bedroom apartment at oluyole wit...
4,"Oke Avenue, Peluseriki area Akala Express Ibad...","₦ 400,000",3 beds,3 baths,3 Toilets,"3 nos of 3 bedroom flat 3T & 3B, Tiled, Wardro..."
...,...,...,...,...,...,...
1995,"Ganiyu bello, felele, off challenge road, ibad...","₦ 250,000",3 beds,2 baths,2 Toilets,"Very clean 3bedroom flat at Ganiyu Bello, fele..."
1996,"Joyce b, ring road Ring Rd Ibadan Oyo","₦ 180,000",1 beds,1 baths,1 Toilets,Very clean Room n parlor self contain at Joyce...
1997,Arapaja area after top one garden Akala Expres...,"₦ 250,000",2 beds,3 baths,3 Toilets,Newly BuiltNewly built of 3 nos of 2 bedroom e...
1998,Apete Eleyele Ibadan Oyo,"₦ 200,000",1 beds,1 baths,2 Toilets,Room and parlour self-contained with guest toi...


In [27]:
abuja_data = parse_listing_data('abuja', 4000000, 3000)
abuja_data

,Area,price_per_Year,beds,baths,toilets,extra
0,"Opposite games village, Kaura (games Village) ...","₦ 2,500,000",3 beds,3 baths,4 Toilets,ServicedA Well Finished and Serviced Three (3)...
1,"Before old zenith bank by gwarinpa entrance, G...","₦ 2,000,000",1 beds,1 baths,2 Toilets,Furnished ServicedA fully furnished and servic...
2,Dakibiyu side of jabi by ics community school ...,"₦ 2,500,000",2 beds,2 baths,3 Toilets,Newly BuiltProperty Description: A Tastefully ...
3,By zartec Wuye Abuja,"₦ 2,500,000",3 beds,3 baths,4 Toilets,Newly BuiltRent: 2.5m S/C: 500. Caution fee 15...
4,By national assembly quarters Apo Abuja,"₦ 1,300,000",1 beds,1 baths,1 Toilets,Newly BuiltStandard 1bedroom block of flat ava...
...,...,...,...,...,...,...
2995,Off Obafemi Awolowo way Utako Abuja,"₦ 3,000,000",3 beds,5 baths,5 Toilets,"ServicedA well Finished, Luxury Built & Beauti..."
2996,Located at Lokogoma district fct Abuja for ren...,"₦ 1,000,000",3 beds,3 baths,4 Toilets,Furnished Serviced Newly Built3 bedroom flat l...
2997,- Lokogoma Abuja,"₦ 200,000",1 beds,1 baths,1 Toilets,To Let newly built a room selfcon located at L...
2998,located at Igbo Agbowa by Abuja off ibeshe Roa...,"₦ 450,000",3 beds,3 baths,3 Toilets,Newly BuiltA Tastefully 2units of 3bedroom fla...


In [28]:
ogun_data = parse_listing_data('ogun', 2000000, 2000)
ogun_data

,Area,price_per_Year,beds,baths,toilets,extra
0,"Royal Estate Warewa, Near Arepo, Ogun State Ar...","₦ 250,000",1 beds,1 baths,1 Toilets,"Royal Estate Warewa, Near Arepo, Ogun State .."
1,Shefun Ibafo Obafemi Owode Ogun,"₦ 250,000",1 beds,1 baths,1 Toilets,FurnishedLovely mini flat at Ibafo Vai ojodu B...
2,Ibafo Obafemi Owode Ogun,"₦ 200,000",0 beds,0 baths,0 Toilets,"Mini flat @ ibafo N200,000 per year.."
3,"@ Kambala, Olambe Yakoyo Alagbole Ojodu Ogun","₦ 150,000",2 beds,2 baths,2 Toilets,Newly BuiltNewly built 2 bedroom flat @ Kambal...
4,Ifesowapo lambe Akute Ifo Ifo Ogun,"₦ 350,000",3 beds,3 baths,3 Toilets,Still available Newly built Executive & Tastef...
...,...,...,...,...,...,...
1681,Toll Gate Ogun,"₦ 300,000",3 beds,3 baths,Toilets,"Lovely 3 bedroom flat at Toll Gate, Ota, fence..."
1682,Magboro Ogun,"₦ 350,000",3 beds,3 baths,Toilets,This 3 bedroom flat is available for rent at M...
1683,Near LAGOS STATE water works Ogun,"₦ 300,000",3 beds,2 baths,Toilets,Newly Built 5 flats Complex at Oke –Aro. Free ...
1684,"Journalist Estate, Phase 1, Arepo Ogun","₦ 600,000",2 beds,2 baths,0 Toilets,Very neat and affordable apartment wit enough ...


In [29]:
ph_data = parse_listing_data('rivers', 3000000, 2500)
ph_data

,Area,price_per_Year,beds,baths,toilets,extra
0,NTA Port Harcourt Rivers,"₦ 650,000",2 beds,3 baths,3 Toilets,2 bedroom Flat / Apartment for rent Nta Port H...
1,Alakahia Port Harcourt Rivers,"₦ 900,000",2 beds,3 baths,3 Toilets,2 bedroom Flat / Apartment for rent Alakahia P...
2,Rumuigbo Port Harcourt Rivers,"₦ 500,000",1 beds,2 baths,2 Toilets,1 bedroom Flat / Apartment for rent Rumuigbo P...
3,Egbelu road Port Harcourt Rivers,"₦ 550,000",2 beds,3 baths,3 Toilets,2 bedroom Flat / Apartment for rent Egbelu Roa...
4,Woji Port Harcourt Rivers,"₦ 1,500,000",2 beds,3 baths,3 Toilets,2 bedroom Flat / Apartment for rent Woji Port ...
...,...,...,...,...,...,...
1713,Boskel Rd Opposite Intels @ Aba Rd Rivers,"₦ 500,000",2 beds,2 baths,Toilets,Twin 2 bedroom flats . Solidly built and strat...
1714,woji Rivers,"₦ 600,000",3 beds,3 baths,Toilets,Fully Fenced Compound | Built in Wardrobes | W...
1715,OGINIGBA Rivers,"₦ 650,000",2 beds,1 baths,Toilets,2BEDROOM FLAT..
1716,Iboliji Estate Rivers,"₦ 550,000",3 beds,2 baths,Toilets,"Spacious, Quiet, Serene Environment Iboloji Es..."
